# Examples

In this notebook, we describe simple examples illustrating the use of `rpa_finder`. See [arXiv:2307.07444](https://arxiv.org/abs/2307.07444) for details.


## Installation Instructions

This notebook uses the following packages:

- `pandas`
- `ipython`
- `rpa_finder`

You can install these packages using the following commands:

```bash
pip install pandas ipython rpa_finder


## Definitions of functions for displaying results

Below we prepare some functions for illustration.

In [6]:
import pandas as pd
from rpa_finder.reaction_system import ReactionSystem
from IPython.display import Markdown, display


def transpose(matrix): # transpose a list of lists
    return list(map(list, zip(*matrix)))


def list_indices(network, input_type='string'):
    system = ReactionSystem(network, input_type)
    bs = system.enumerate_buffering_structures()
    
    influence_indices = [system.compute_influence_index(b) for b in bs]
    num_emergent_cycles = [ len(system.get_emergent_cycles(b)) for b in bs]
    num_emergent_conserved_quantities = [ len(system.get_emergent_conserved_quantities(b)) for b in bs]
    num_lost_conserved_quantities = [ len(system.get_lost_conserved_quantities(b)) for b in bs]    

    bs_name = [ system.bs_to_name(b) for b in bs]
    bs_T = transpose(bs_name)

    bs_data = {
        'species' : bs_T[0],
        'reactions' : bs_T[1],
        'influence index' : influence_indices,
        'em. cycles' : num_emergent_cycles,
        'lost cons. quant.' : num_lost_conserved_quantities,
        'em. cons. quant.' : num_emergent_conserved_quantities
    }

    title_bs = 'Buffering structures'
    df_bs = pd.DataFrame(bs_data)
    styled_df_bs = df_bs.style.set_table_attributes('style="font-size: 13px"').set_caption(title_bs)

    display(styled_df_bs)


def display_integrators(integrators):
    for i in range(4):
        display( Markdown("$\\frac{d}{dt}" + integrators[2*i] + "=" + integrators[2*i+1] + "$") )
    return




## Example 1


We consider a reaction network with the following list of reactions:

$e_1: \emptyset \to v_1$

$e_2: v_1 \to v_2$ 

$e_3: v_2 \to v_3$

$e_4: v_3 \to v_1$

$e_5: v_2 \to \emptyset$




A `ReactionSystem` object corresponding to this network can be created as follows:

In [7]:
network1 = """
"", "v1"
"v1", "v2"
"v2", "v3"
"v3", "v1"
"v2", ""
"""

r_system1 = ReactionSystem(network1)


The labeled buffering structures for this network is computed by `enumerate_labeled_buffering_structures()` as follows:

In [8]:
lbs_list_1 = r_system1.enumerate_labeled_buffering_structures()

for lbs in lbs_list_1:
    print(lbs)


[[0], [0, 1, 2], [0, 1, 2, 3, 4], []]
[[1], [0], [], [1]]
[[2], [0, 2], [1, 2, 3], []]
[[3], [2], [], [3]]
[[4], [0, 1, 2], [1, 2, 3], [4]]


Note that the indices of species and reactions start with zero.
To show them using names for species, one can use the method `lbs_to_name`:

In [9]:
for lbs in lbs_list_1:
    print(r_system1.lbs_to_name(lbs))



[[0], ['v1', 'v2', 'v3'], [0, 1, 2, 3, 4], []]
[[1], ['v1'], [], [1]]
[[2], ['v1', 'v3'], [1, 2, 3], []]
[[3], ['v3'], [], [3]]
[[4], ['v1', 'v2', 'v3'], [1, 2, 3], [4]]


Let us obtain the integrator equations for the RPA property represented by a labeled buffering structure.


In [10]:
num = 1

display_integrators(
    r_system1.find_integrators_from_lbs(lbs_list_1[num], symbol_mode='name')
)



$\frac{d}{dt}\left[\begin{matrix}\end{matrix}\right]=\left[\begin{matrix}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}v_{1} + v_{2}\\v_{3}\end{matrix}\right]=\left[\begin{matrix}r_{1} - r_{3} + r_{4} - r_{5}\\r_{3} - r_{4}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}- v_{1}\end{matrix}\right]=\left[\begin{matrix}- r_{1} + r_{2} - r_{4}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}v_{2}\end{matrix}\right]=\left[\begin{matrix}r_{2} - r_{3} - r_{5}\end{matrix}\right]$

For each buffering structure, let us list the influence index, the number of emergent cycles, lost conserved quantities, and emergent conserved quantities:

In [11]:
list_indices(network1)

,species,reactions,influence index,em. cycles,lost cons. quant.,em. cons. quant.
0,"['v1', 'v2', 'v3']","[0, 1, 2, 3, 4]",0,0,0,0
1,['v1'],[1],0,0,0,0
2,"['v1', 'v3']","[1, 2, 3]",0,0,0,0
3,['v3'],[3],0,0,0,0
4,"['v1', 'v2', 'v3']","[1, 2, 3, 4]",0,0,0,0


## Example 2


$e_1: \emptyset \to v_1$

$e_2: v_1 \to v_2$

$e_3: v_2 \to \emptyset$

$e_4: v_1 + v_2 \to v_3 + v_4$

$e_5: v_3 + v_4 \to v_1 + v_2$





In [12]:
from rpa_finder.reaction_system import ReactionSystem
from rpa_finder.reaction_system import from_lbs_to_bs

network_ex2 = """
"", "v1"
"v1", "v2"
"v2", ""
"v1"+"v2", "v3"+"v4"
"v3"+"v4", "v1"+"v2"
"""

r_sys_cons = ReactionSystem( network_ex2 )
lbs_cons = r_sys_cons.enumerate_labeled_buffering_structures()

for lbs in lbs_cons:
    print(lbs)


[[0], [0, 1, 2, 3], [0, 1, 2, 3, 4], []]
[[1], [0, 2, 3], [3, 4], [1]]
[[2], [1, 2, 3], [3, 4], [2]]
[[3], [2, 3], [3, 4], []]
[[4, 5], [2, 3], [], [4]]


In [13]:
num = 4

display_integrators(
    r_sys_cons.find_integrators_from_lbs(lbs_cons[num], symbol_mode='name')
)

$\frac{d}{dt}\left[\begin{matrix}\end{matrix}\right]=\left[\begin{matrix}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}v_{1} + \frac{v_{3}}{2} + \frac{v_{4}}{2}\\v_{2} + \frac{v_{3}}{2} + \frac{v_{4}}{2}\end{matrix}\right]=\left[\begin{matrix}r_{1} - r_{2}\\r_{2} - r_{3}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}- \frac{v_{3}}{2} - \frac{v_{4}}{2}\end{matrix}\right]=\left[\begin{matrix}- r_{4} + r_{5}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}\frac{v_{1}}{2} + \frac{v_{2}}{2}\end{matrix}\right]=\left[\begin{matrix}\frac{r_{1}}{2} - \frac{r_{3}}{2} - r_{4} + r_{5}\end{matrix}\right]$

In [14]:
list_indices( network_ex2 )


,species,reactions,influence index,em. cycles,lost cons. quant.,em. cons. quant.
0,"['v1', 'v2', 'v3', 'v4']","[0, 1, 2, 3, 4]",0,0,0,0
1,"['v1', 'v3', 'v4']","[1, 3, 4]",0,0,0,0
2,"['v2', 'v3', 'v4']","[2, 3, 4]",0,0,0,0
3,"['v3', 'v4']","[3, 4]",0,0,0,0
4,"['v3', 'v4']",[4],0,0,0,0


## Example 3

Let us discuss a reaction system with antithetic integral control.

The list of reactions is as follows:

$e_1 : z_1 \to  z_1+x$

$e_2 : z_1 + z_2 \to \emptyset$

$e_3 : \emptyset \to z_1$

$e_4 : 2x \to 2x + z_2 $

$e_5 : x \to \emptyset $

$e_6 : y \to y+z_1 $

$e_7 : \emptyset \to y $

$e_8 : y \to \emptyset $

In [15]:
network_antithetic = """
"z1", "z1"+"x"
"z1"+"z2", ""
"", "z1"
2"x", 2"x"+"z2"
"x", ""
"y", "y"+"z1"
"", "y"
"y", ""
"""

r_sys_antithetic = ReactionSystem(network_antithetic)
lbs_antithetic = r_sys_antithetic.enumerate_labeled_buffering_structures()

for lbs in lbs_antithetic:
    print(lbs)


[[0], [2, 3], [], [0, 1]]
[[1], [3], [], [1]]
[[2], [0, 2, 3], [0, 1, 2, 3, 4], []]
[[3], [0, 2, 3], [0, 4], [1, 3]]
[[4], [2, 3], [0, 4], [1]]
[[5], [0, 2, 3], [0, 1, 3, 4, 5], []]
[[6], [0, 1, 2, 3], [0, 1, 3, 4, 5, 6, 7], []]
[[7], [0, 1, 2, 3], [0, 1, 3, 4, 5], [7]]


In [16]:
num = 0

display_integrators(
    r_sys_antithetic.find_integrators_from_lbs(lbs_antithetic[num], symbol_mode='name')
)

$\frac{d}{dt}\left[\begin{matrix}- z_{1} + z_{2}\end{matrix}\right]=\left[\begin{matrix}- r_{3} + r_{4} - r_{6}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}y\end{matrix}\right]=\left[\begin{matrix}r_{7} - r_{8}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}- \frac{z_{1}}{2} - \frac{z_{2}}{2}\end{matrix}\right]=\left[\begin{matrix}r_{2} - \frac{r_{3}}{2} - \frac{r_{4}}{2} - \frac{r_{6}}{2}\end{matrix}\right]$

$\frac{d}{dt}\left[\begin{matrix}x\end{matrix}\right]=\left[\begin{matrix}r_{1} - r_{5}\end{matrix}\right]$

In [17]:
list_indices( network_antithetic )


,species,reactions,influence index,em. cycles,lost cons. quant.,em. cons. quant.
0,"['z1', 'z2']","[0, 1]",0,1,0,1
1,['z2'],[1],0,0,0,0
2,"['x', 'z1', 'z2']","[0, 1, 2, 3, 4]",0,0,0,0
3,"['x', 'z1', 'z2']","[0, 1, 3, 4]",0,0,0,0
4,"['z1', 'z2']","[0, 1, 4]",0,1,0,1
5,"['x', 'z1', 'z2']","[0, 1, 3, 4, 5]",0,0,0,0
6,"['x', 'y', 'z1', 'z2']","[0, 1, 3, 4, 5, 6, 7]",0,0,0,0
7,"['x', 'y', 'z1', 'z2']","[0, 1, 3, 4, 5, 7]",0,0,0,0


The buffering structures with nonzero numbers of emergent conserved quantities have antithetic integral control.